In [1]:
ENV["PYTHONPATH"]="/home/gridsan/aligho/.local/lib/python3.8/site-packages/PyNormaliz-2.15-py3.8-linux-x86_64.egg";

In [2]:
using PyCall, HDF5, Crystalline, MPBUtils, SymmetryBases, StaticArrays, PyPlot, LinearAlgebra, JLD2, 
Statistics, DelimitedFiles
np = pyimport("numpy");
include("../get-freqs-symeigs.jl")

logs_to_symeigs (generic function with 1 method)

In [34]:
diffs_dict = Dict{Tuple{Int64, Int64, String}, Vector{Tuple{Integer, Integer, Vector{Float64}}}}()
failed_loads_dict = Dict{Tuple{Int64, Int64, String}, Vector{Int64}}()
for sgnum in [2, 10, 13, 16]
    println("sgnum: $sgnum")
    for epsid in 1:3
        println("epsid: $epsid"); flush(stdout)
        for mode in ["te", "tm"]
            println("mode: $mode"); flush(stdout)
            log_filename = "sg$sgnum-epsid$epsid-res64-$mode-log.jld2"
            disp_log_dir = "./logs/"
            symeigs_log_dir = "../symeigs/logs/"
            loaded_logs_disp = load(disp_log_dir * log_filename)["logsv"]
            loaded_logs_symeigs = load(symeigs_log_dir * log_filename)["logsv"];
            idx_diffs = Vector{Tuple{Integer, Integer, Vector{Float64}}}()
            failed_loads = Int64[]
            for idx in 1:10000
                try 
                    max_diffs_k = Float64[]
                    (idx % 1000 == 0) && (println("idx: $idx"); flush(stdout))
                    dispersion_str = logs_to_dispersion(loaded_logs_disp[idx])
                    symeigs_str = logs_to_dispersion(loaded_logs_symeigs[idx]);
                    
                    dispersion_io = IOBuffer(dispersion_str)
                    symeigs_io = IOBuffer(symeigs_str)
                    
                    dispersion_dlm = readdlm(dispersion_io, ',')::Matrix{Float64}
                    symeigs_dlm = readdlm(symeigs_io, ',')::Matrix{Float64}
                    
                    dispersion_kpts = dispersion_dlm[:, 2:4]
                    symeigs_kpts = symeigs_dlm[:, 2:4]
                    
                    dispersion_freqs = sort(dispersion_dlm[:, 6:end], dims = 2)
                    symeigs_freqs = sort(symeigs_dlm[:, 6:end], dims = 2)
                    
                    for (kpt_idx, (kpt, symeigs_freqs_at_k)) in enumerate(zip(eachrow(symeigs_kpts), eachrow(symeigs_freqs)))
                        disp_kpt_symeigs_kpts_diffs = [norm(kpt .- disp_kpt) for disp_kpt in eachrow(dispersion_kpts)]
                        disp_idx = argmin(disp_kpt_symeigs_kpts_diffs)
                        iszero(minimum(disp_kpt_symeigs_kpts_diffs)) || continue
                        #push!(max_diffs_k, maximum(abs.(dispersion_freqs[disp_idx, :] - symeigs_freqs_at_k)))
                        percent_changes_band = replace(x-> isnan(x) ? 0 : x, (dispersion_freqs[disp_idx, :] - symeigs_freqs_at_k) ./ symeigs_freqs_at_k)
                        push!(idx_diffs, (idx, kpt_idx, percent_changes_band[1:20]))
                    end
                    #push!(max_diffs, maximum(max_diffs_k))
                catch
                    push!(failed_loads, idx)
                end
            end
           # push!(max_diffs_dict, (sgnum, epsid, mode) => (argmax(max_diffs), maximum(max_diffs)))
            push!(failed_loads_dict, (sgnum, epsid, mode) => failed_loads)
            push!(diffs_dict, (sgnum, epsid, mode) => idx_diffs)
        end
    end
end

sgnum: 2
epsid: 1
mode: te
idx: 1000
idx: 2000
idx: 3000
idx: 4000
idx: 5000
idx: 6000
idx: 7000
idx: 8000
idx: 9000
idx: 10000
mode: tm
idx: 1000
idx: 2000
idx: 3000
idx: 4000
idx: 5000
idx: 6000
idx: 7000
idx: 8000
idx: 9000
idx: 10000
epsid: 2
mode: te
idx: 1000
idx: 2000
idx: 3000
idx: 4000
idx: 5000
idx: 6000
idx: 7000
idx: 8000
idx: 9000
idx: 10000
mode: tm
idx: 1000
idx: 2000
idx: 3000
idx: 4000
idx: 5000
idx: 6000
idx: 7000
idx: 8000
idx: 9000
idx: 10000
epsid: 3
mode: te
idx: 1000
idx: 2000
idx: 3000
idx: 4000
idx: 5000
idx: 6000
idx: 7000
idx: 8000
idx: 9000
idx: 10000
mode: tm
idx: 1000
idx: 2000
idx: 3000
idx: 4000
idx: 5000
idx: 6000
idx: 7000
idx: 8000
idx: 9000
idx: 10000
sgnum: 10
epsid: 1
mode: te
idx: 1000
idx: 2000
idx: 3000
idx: 4000
idx: 5000
idx: 6000
idx: 7000
idx: 8000
idx: 9000
idx: 10000
mode: tm
idx: 1000
idx: 2000
idx: 3000
idx: 4000
idx: 5000
idx: 6000
idx: 7000
idx: 8000
idx: 9000
idx: 10000
epsid: 2
mode: te
idx: 1000
idx: 2000
idx: 3000
idx: 4000
idx: 50

In [35]:
for sgnum in [2, 10, 13, 16]
    println("sgnum: $sgnum"); flush(stdout)
    for mode in ["te", "tm"]
        println("mode: $mode"); flush(stdout)
        for epsid in 1:3
            println("epsid: $epsid"); flush(stdout)
            max_diff = maximum([maximum(abs.(x[3])) for x in diffs_dict[(sgnum, epsid, mode)]])
            println("Max percentage difference: $(round(max_diff*100, digits=3))", "\n")
        end
    end
end

sgnum: 2
mode: te
epsid: 1
Max percentage difference: 0.013

epsid: 2
Max percentage difference: 0.011

epsid: 3
Max percentage difference: 0.015

mode: tm
epsid: 1
Max percentage difference: 0.011

epsid: 2
Max percentage difference: 0.012

epsid: 3
Max percentage difference: 0.01

sgnum: 10
mode: te
epsid: 1
Max percentage difference: 0.015

epsid: 2
Max percentage difference: 0.015

epsid: 3
Max percentage difference: 0.016

mode: tm
epsid: 1
Max percentage difference: 0.014

epsid: 2
Max percentage difference: 0.015

epsid: 3
Max percentage difference: 0.015

sgnum: 13
mode: te
epsid: 1
Max percentage difference: 0.014

epsid: 2
Max percentage difference: 0.016

epsid: 3
Max percentage difference: 0.016

mode: tm
epsid: 1
Max percentage difference: 0.014

epsid: 2
Max percentage difference: 0.015

epsid: 3
Max percentage difference: 0.019

sgnum: 16
mode: te
epsid: 1
Max percentage difference: 0.015

epsid: 2
Max percentage difference: 0.018

epsid: 3
Max percentage difference: 0.0

In [36]:
failed_loads_dict

Dict{Tuple{Int64, Int64, String}, Vector{Int64}} with 24 entries:
  (13, 1, "tm") => []
  (16, 2, "te") => []
  (2, 1, "tm")  => []
  (10, 1, "tm") => []
  (16, 3, "te") => []
  (13, 2, "tm") => []
  (2, 2, "tm")  => []
  (10, 2, "tm") => []
  (13, 3, "tm") => []
  (16, 1, "tm") => []
  (2, 3, "tm")  => []
  (10, 3, "tm") => []
  (16, 2, "tm") => []
  (13, 1, "te") => []
  (16, 3, "tm") => []
  (2, 1, "te")  => [3404, 3412]
  (10, 1, "te") => []
  (13, 2, "te") => []
  (2, 2, "te")  => []
  (10, 2, "te") => []
  (13, 3, "te") => []
  (16, 1, "te") => [3803, 3806, 3807]
  (2, 3, "te")  => []
  (10, 3, "te") => []

In [37]:
# Below we check for the few lattices where log files got messed up: 

In [62]:
for ((sgnum, epsid, mode), failed_idxs) in failed_loads_dict
    isempty(failed_idxs) && continue
    println("failed idxs for sgnum=$sgnum, epsid=$epsid, and mode=$mode: ", failed_idxs); flush(stdout)
    log_filename = "sg$sgnum-epsid$epsid-res64-$mode-log.jld2"
    symeigs_log_dir = "../symeigs/logs/"
    loaded_logs_symeigs = load(symeigs_log_dir * log_filename)["logsv"];
    for failed_idx in failed_idxs
        dispersion_dlm = readdlm("./output/sg$sgnum/eps$epsid/$mode/dim2-sg$sgnum-$(failed_idx+10000*(epsid-1))-res64-$mode-dispersion.out", ',')::Matrix{Float64}
        symeigs_str = logs_to_dispersion(loaded_logs_symeigs[failed_idx]);
        symeigs_io = IOBuffer(symeigs_str)
        symeigs_dlm = readdlm(symeigs_io, ',')::Matrix{Float64}
        dispersion_kpts = dispersion_dlm[:, 2:3]
        symeigs_kpts = symeigs_dlm[:, 2:3]
        dispersion_freqs = sort(dispersion_dlm[:, 6:end], dims = 2)
        symeigs_freqs = sort(symeigs_dlm[:, 6:end], dims = 2)
        for (kpt_idx, (kpt, symeigs_freqs_at_k)) in enumerate(zip(eachrow(symeigs_kpts), eachrow(symeigs_freqs)))
            disp_kpt_symeigs_kpts_diffs = [norm(kpt .- disp_kpt) for disp_kpt in eachrow(dispersion_kpts)]
            disp_idx = argmin(disp_kpt_symeigs_kpts_diffs)
            iszero(minimum(disp_kpt_symeigs_kpts_diffs)) || continue
            #push!(max_diffs_k, maximum(abs.(dispersion_freqs[disp_idx, :] - symeigs_freqs_at_k)))
            percent_changes_band = replace(x-> isnan(x) ? 0 : x, (dispersion_freqs[disp_idx, :] - symeigs_freqs_at_k) ./ symeigs_freqs_at_k)
            println(round.(percent_changes_band, digits=3))
        end    
    end
end

failed idxs for sgnum=2, epsid=1, and mode=te: [3404, 3412]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0